In [1]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [28]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [31]:
data.loc[data['2030val'].apply(pd.isna), '2030val'] = 0
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [32]:
# 격자 데이터
data_gid = data[['gid', 'geometry']]

In [33]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


# 횡단보도 부착대

In [44]:
data19 = gpd.read_file("output/서초강남횡단보도부착대.geojson")

,STAT_CDE,MGRNU,A062_KND_C,SPRT_STD_C,EVE_CDE,SPRT_HOTDI,CSS_CDE,ESB_YMD,CAE_YMD,OD_PE_CDE,...,field_66,field_67,field_68,field_69,field_70,field_71,field_72,field_73,gid,geometry
0,1,02-0000178068,1,6,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204221.148207307,544841.555069,204219.603122,544836.799786406)),252.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04774 37.50303)
1,1,02-0000178067,1,7,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204212.929621534,544827.997402,204217.657214,544829.625242717)),19.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04765 37.50291)
2,1,02-0000178067,1,7,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204212.929621534,544827.997402,204211.219521,544832.695865049)),110.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04765 37.50291)
3,1,02-0000109590,1,7,1,2,288.0,20131220,20131220,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(205374.987783456,544360.323503,205374.986073,544360.328201166)),110.0,1,0.0,ㅼ궗611444,MULTIPOINT (127.06078 37.49869)
4,1,02-0000109590,1,7,1,2,288.0,20131220,20131220,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(205374.987783456,544360.323503,205371.045736,544370.592887394)),111.0,3,0.0,ㅼ궗611444,MULTIPOINT (127.06078 37.49869)


In [45]:
gid_list = []
for i in tqdm(range(data19.shape[0])):
    point = data19['geometry'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██| 5345/5345 [10:08<00:00,  8.79it/s]


In [46]:
data19['gid'] = gid_list
data19['TL_cnt']=1
result19 = data19[['TL_cnt', 'gid']].groupby(['gid']).sum()
result19.reset_index(inplace=True)
data19 =result19
data19.head()

,gid,TL_cnt
0,다사541420,21
1,다사541421,1
2,다사541424,6
3,다사541425,4
4,다사541427,5


In [47]:
data19

,gid,TL_cnt
0,다사541420,21
1,다사541421,1
2,다사541424,6
3,다사541425,4
4,다사541427,5
...,...,...
1290,다사663411,2
1291,다사663413,2
1292,다사664409,3
1293,다사664410,2


In [48]:
data19.to_csv("output/서초강남횡단보도부착대전처리완료.csv")

# 버스정류장 전처리

In [51]:
data1 = pd.read_csv("output/강남구서초구정류장저녁승차.csv")
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수
0,22001,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227
1,22002,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488
...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129


In [52]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [53]:
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,bus_cnt,point
0,22001,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227,1,POINT (127.03834 37.47851)
1,22002,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091,1,POINT (127.03868 37.47777)
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889,1,POINT (127.03585 37.48252)
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872,1,POINT (127.03683 37.48187)
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667,1,POINT (127.09474 37.48349)
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667,1,POINT (127.08286 37.48897)
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667,1,POINT (127.10154 37.46460)
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129,1,POINT (127.08768 37.49066)


In [54]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██| 1166/1166 [02:23<00:00,  8.15it/s]


In [55]:
data1['gid'] = gid_list
data1['TL_cnt']=1
result1 = data1[['TL_cnt', 'gid']].groupby(['gid']).sum()
result1.reset_index(inplace=True)
data1 =result1


In [56]:
data1

,gid,TL_cnt
0,Na,1
1,다사540452,2
2,다사541419,1
3,다사541421,1
4,다사541423,1
...,...,...
774,다사655420,2
775,다사658421,2
776,다사660408,1
777,다사662409,1


In [60]:
data1[data1["gid"] == "Na"]

,gid,TL_cnt
0,Na,1


In [61]:
data2 = data1.drop(index=0, axis=0)

In [62]:
data2

,gid,TL_cnt
1,다사540452,2
2,다사541419,1
3,다사541421,1
4,다사541423,1
5,다사542418,1
...,...,...
774,다사655420,2
775,다사658421,2
776,다사660408,1
777,다사662409,1


In [63]:
data2.to_csv("output/서초강남버스정류장전처리완료.csv")

# 지하철 역 전처리

In [65]:
data3 = pd.read_csv("output/서초강남지하철승하차.csv")
data3

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [66]:
data3 = data3[["역이름","위도","경도"]]

In [70]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [68]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-68-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-68-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-68-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [69]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [71]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████| 47/47 [00:06<00:00,  7.40it/s]


In [72]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [73]:
data3

,gid,TL_cnt
0,Na,3
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1


In [74]:
data3[data3["gid"] == "Na"]

,gid,TL_cnt
0,Na,3


In [75]:
data4 = data3.drop(index=0, axis=0)

In [76]:
data4

,gid,TL_cnt
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1
10,다사564437,1


In [77]:
data4.to_csv("output/서초강남지하철역전처리완료.csv")

# 상권 전처리

In [90]:
store = pd.read_csv("output/서초강남상권.csv")
store

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137837,6572.0,NaN,1,102,NaN,126.991394,37.488375,NaN,2021-01-09
1,18,17174636,호랑이가만든족발,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,135513,6148.0,NaN,NaN,NaN,02-6465-1314,127.044182,37.504996,1943.0,2021-01-09
2,19,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137869,6709.0,NaN,1,NaN,02-583-5658,127.009382,37.483436,NaN,2020-11-27
3,20,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,민속주점,...,135894,6023.0,NaN,1,NaN,02-544-8819,127.031012,37.528073,2053.0,2017-08-27
4,23,22795034,든든한우家,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137070,6652.0,NaN,2,NaN,02-586-4000,127.012363,37.486335,1845.0,2017-09-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47854,316900,17456528,크린스퀘어,NaN,F,생활서비스,F02,세탁/가사서비스,F02A01,세탁소/빨래방,...,135877,6158.0,101,1,B110,NaN,127.055465,37.508079,NaN,2021-02-11
47855,316902,17431606,안젤라화이트,NaN,F,생활서비스,F01,이/미용/건강,F01A03,비만/피부관리,...,137817,6677.0,NaN,2,201,NaN,126.982924,37.482598,NaN,2021-01-11
47856,316913,17456260,서울방역공사,NaN,F,생활서비스,F02,세탁/가사서비스,F02A05,청소/소독,...,137843,6682.0,NaN,1,NaN,NaN,126.993660,37.484673,NaN,2021-02-11
47857,316917,17446911,서래와규짱,서래본점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137831,6585.0,NaN,NaN,NaN,NaN,126.991262,37.492275,NaN,2021-01-26


In [91]:
store.상권업종대분류명.unique()

array(['음식', '부동산', '소매', '생활서비스', '관광/여가/오락', '학문/교육', '숙박', '스포츠'],
      dtype=object)

In [92]:
store.상권업종중분류명.unique()

array(['한식', '유흥주점', '부동산중개', '의복의류', '선물/팬시/기념품', '사무/문구/컴퓨터', '자동차/이륜차',
       'PC/오락/당구/볼링등', '기타서비스업', '학원-보습교습입시', '사진/광학/정밀기기소매', '종합소매점',
       '학문교육기타', '양식', '세탁/가사서비스', '학원-음악미술무용', '분식', '호텔/콘도', '건강/미용식품',
       '음/식료품소매', '물품기기대여', '책/서적/도서', '요가/단전/마사지', '가방/신발/액세서리', '화장품소매',
       '이/미용/건강', '닭/오리요리', '가전제품소매', '주택수리', '개인/가정용품수리', '일식/수산물',
       '학원기타', '연극/영화/극장', '가정/주방/인테리어', '패스트푸드', '제과제빵떡케익', '커피점/카페',
       '사진', '가구소매', '학원-어학', '자동차/자동차용품', '철물/난방/건설자재소매', '유아교육',
       '운동/경기용품소매', '분양', '주유소/충전소', '광고/인쇄', '취미/오락관련소매',
       '예술품/골동품/수석/분재', '예식/의례/관혼상제', '무도/유흥/가무', '기타판매업', '대행업',
       '도서관/독서실', '인력/고용/용역알선', '시계/귀금속소매', '의약/의료품소매', '법무세무회계',
       '종교용품판매', '학원-예능취미체육', '평가/개발/관리', '중식', '운송/배달/택배', '개인서비스',
       '기타음식업', '별식/퓨전요리', '유아용품', '경마/경륜/성인오락', '부페', '학원-자격/국가고시',
       '놀이/여가/취미', '모텔/여관/여인숙', '페인트/유리제품소매', '학원-창업취업취미', '중고품소매/교환',
       '장례/묘지', '운영관리시설', '대중목욕탕/휴게', '행사/이벤트', '음식배달서비스', '민박/하숙',
       '실내운동시설', '학원-컴퓨터', '캠프

### '음식', '부동산', '소매', '생활서비스', '관광/여가/오락', '학문/교육', '숙박', '스포츠' : 상권업종대분류명으로 나눈 후에 전처리 

In [78]:
data3 = pd.read_csv("output/서초강남상권.csv")
data3

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137837,6572.0,NaN,1,102,NaN,126.991394,37.488375,NaN,2021-01-09
1,18,17174636,호랑이가만든족발,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,135513,6148.0,NaN,NaN,NaN,02-6465-1314,127.044182,37.504996,1943.0,2021-01-09
2,19,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137869,6709.0,NaN,1,NaN,02-583-5658,127.009382,37.483436,NaN,2020-11-27
3,20,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,민속주점,...,135894,6023.0,NaN,1,NaN,02-544-8819,127.031012,37.528073,2053.0,2017-08-27
4,23,22795034,든든한우家,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137070,6652.0,NaN,2,NaN,02-586-4000,127.012363,37.486335,1845.0,2017-09-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47854,316900,17456528,크린스퀘어,NaN,F,생활서비스,F02,세탁/가사서비스,F02A01,세탁소/빨래방,...,135877,6158.0,101,1,B110,NaN,127.055465,37.508079,NaN,2021-02-11
47855,316902,17431606,안젤라화이트,NaN,F,생활서비스,F01,이/미용/건강,F01A03,비만/피부관리,...,137817,6677.0,NaN,2,201,NaN,126.982924,37.482598,NaN,2021-01-11
47856,316913,17456260,서울방역공사,NaN,F,생활서비스,F02,세탁/가사서비스,F02A05,청소/소독,...,137843,6682.0,NaN,1,NaN,NaN,126.993660,37.484673,NaN,2021-02-11
47857,316917,17446911,서래와규짱,서래본점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137831,6585.0,NaN,NaN,NaN,NaN,126.991262,37.492275,NaN,2021-01-26


In [79]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [80]:
data3

,상권업종대분류명,위도,경도
0,음식,37.488375,126.991394
1,음식,37.504996,127.044182
2,음식,37.483436,127.009382
3,음식,37.528073,127.031012
4,음식,37.486335,127.012363
...,...,...,...
47854,생활서비스,37.508079,127.055465
47855,생활서비스,37.482598,126.982924
47856,생활서비스,37.484673,126.993660
47857,음식,37.492275,126.991262


In [81]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-81-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-81-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-81-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [82]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,음식,37.488375,126.991394,1,POINT (126.99139 37.48837)
1,음식,37.504996,127.044182,1,POINT (127.04418 37.50500)
2,음식,37.483436,127.009382,1,POINT (127.00938 37.48344)
3,음식,37.528073,127.031012,1,POINT (127.03101 37.52807)
4,음식,37.486335,127.012363,1,POINT (127.01236 37.48634)
...,...,...,...,...,...
47854,생활서비스,37.508079,127.055465,1,POINT (127.05546 37.50808)
47855,생활서비스,37.482598,126.982924,1,POINT (126.98292 37.48260)
47856,생활서비스,37.484673,126.993660,1,POINT (126.99366 37.48467)
47857,음식,37.492275,126.991262,1,POINT (126.99126 37.49227)


In [89]:
data3.상권업종대분류명.unique()

array(['음식', '부동산', '소매', '생활서비스', '관광/여가/오락', '학문/교육', '숙박', '스포츠'],
      dtype=object)

In [83]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

 31%|▎| 14935/47859 [37:29<1:22:39,  6.64it


KeyboardInterrupt: 

In [72]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [73]:
data3

,gid,TL_cnt
0,Na,3
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1


In [74]:
data3[data3["gid"] == "Na"]

,gid,TL_cnt
0,Na,3


In [75]:
data4 = data3.drop(index=0, axis=0)

In [76]:
data4

,gid,TL_cnt
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1
10,다사564437,1


In [77]:
data4.to_csv("output/서초강남상권전처리완료.csv")

# 과속방지턱 전처리

In [65]:
data3 = pd.read_csv("output/서초강남과속방지턱.csv")
data3

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [66]:
data3 = data3[["역이름","위도","경도"]]

In [70]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [68]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-68-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-68-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-68-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [69]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [71]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████| 47/47 [00:06<00:00,  7.40it/s]


In [72]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [73]:
data3

,gid,TL_cnt
0,Na,3
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1


In [74]:
data3[data3["gid"] == "Na"]

,gid,TL_cnt
0,Na,3


In [75]:
data4 = data3.drop(index=0, axis=0)

In [76]:
data4

,gid,TL_cnt
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1
10,다사564437,1


In [77]:
data4.to_csv("output/서초강남과속방지턱전처리완료.csv")

# 서초 강남 사고 전처리

In [65]:
data3 = pd.read_csv("output/서초강남PM사고위치.csv")
data3

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [66]:
data3 = data3[["역이름","위도","경도"]]

In [70]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [68]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-68-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-68-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-68-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [69]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [71]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████| 47/47 [00:06<00:00,  7.40it/s]


In [72]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [73]:
data3

,gid,TL_cnt
0,Na,3
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1


In [74]:
data3[data3["gid"] == "Na"]

,gid,TL_cnt
0,Na,3


In [75]:
data4 = data3.drop(index=0, axis=0)

In [76]:
data4

,gid,TL_cnt
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1
10,다사564437,1


In [77]:
data4.to_csv("output/서초강남PM교통사고전처리완료.csv")

# 자전거도로 전처리

In [88]:
data19 = gpd.read_file("data/자전거 이용 데이터/전처리/출근시간교차영역.geojson")
data19

DriverError: Failed to read GeoJSON data